### Notebook to make plots of surface/bottom isohalines right as the instabilities reach the boundary.

In [1]:
#Packages 
import numpy as np
import xgcm
from xgcm import Grid
import xarray as xr
import xroms
from datetime import datetime

import glob
from xhistogram.xarray import histogram
import cmocean.cm as cmo
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.ticker as tick
from matplotlib.dates import DateFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator
from xhistogram.xarray import histogram
from datetime import timedelta
import time

import warnings
warnings.filterwarnings("ignore") #The chaotic option, used to suppress issues with cf_time with xroms 

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import pyplot as plt, patches

### Open model output

In [2]:
def open_roms(path):
    ds1 = xroms.open_netcdf(path)
    ds1, grid1 = xroms.roms_dataset(ds1)
    return ds1, grid1

paths = ['/d2/home/dylan/idealized_nummix/shelf_mpdata_uwind_zerop1_dt_30_60d_avg.nc',
         '/d2/home/dylan/idealized_nummix/shelf_u3hc4_uwind_zerop1_dt_30_60d_avg.nc',
         '/d2/home/dylan/idealized_nummix/shelf_hsimt_uwind_zerop1_dt_30_60d_avg.nc']

def open_roms(path):
    ds1 = xroms.open_netcdf(path)
    ds1, grid1 = xroms.roms_dataset(ds1)
    ds1 = ds1.isel(ocean_time = slice(0,604))
    return ds1, grid1

ds = []
grid = []
for i in range(len(paths)):
    ds1, grid1 = open_roms(paths[i])
    ds.append(ds1)
    grid.append(grid1)
    # print('iter complete')

### Check the time at t = 600 to make sure its day 26
Also calculate the range for the colorbar 

In [3]:
t = 600 
print(ds[0].ocean_time[t].values)
print(ds[0].salt[t,-1].max().values)
print(ds[1].salt[t,-1].max().values)
print(ds[2].salt[t,-1].max().values)
print(ds[0].salt[t,-1].min().values)
print(ds[1].salt[t,-1].min().values)
print(ds[2].salt[t,-1].min().values)

0001-01-26 00:30:00
34.99998
35.01984
34.999985
27.913433
27.838411
27.884632


### Plot the surface and bottom salinity, overlay the contours 

In [ ]:
nrows = 2; ncols = 3
fig, ax = plt.subplots(nrows,ncols, 
                       figsize = ((7/2)*2.55,(8.66/3)*2), #Have to brute force the size
                       constrained_layout = True)
#Surface
cs = ax[0,0].pcolormesh(ds[0].x_rho/1000,ds[0].y_rho/1000, ds[0].salt[t,-1], 
                        vmin = 28, vmax = 35, cmap = cm)
cc = ax[0,0].contour(ds[0].x_rho/1000,ds[0].y_rho/1000, ds[0].salt[t,-1], 
                     levels = np.arange(28,36,0.5), colors = 'k', linewidths = 0.8,
                     add_colorbar = False)
ax[0,1].pcolormesh(ds[1].x_rho/1000,ds[1].y_rho/1000, ds[1].salt[t,-1], 
                   vmin = 28, vmax = 35, cmap = cm)
ax[0,1].contour(ds[1].x_rho/1000,ds[1].y_rho/1000, ds[1].salt[t,-1], 
               levels = np.arange(28,35.5,0.5), colors = 'k', linewidths = 0.8,
               add_colorbar = False)
ax[0,2].pcolormesh(ds[2].x_rho/1000,ds[2].y_rho/1000, ds[2].salt[t,-1], 
                   vmin = 28, vmax = 35, cmap = cm)
ax[0,2].contour(ds[2].x_rho/1000,ds[2].y_rho/1000, ds[2].salt[t,-1], 
                levels = np.arange(28,36,0.5), colors = 'k', linewidths = 0.8,
                add_colorbar = False)
#Bottom
cs1 = ax[1,0].pcolormesh(ds[0].x_rho/1000,ds[0].y_rho/1000, ds[0].salt[t,0], 
                         vmin = 28, vmax = 35, cmap = cm)
cc = ax[1,0].contour(ds[0].x_rho/1000,ds[0].y_rho/1000, ds[0].salt[t,0], 
                     levels = np.arange(28,36,0.5), colors = 'k', linewidths = 0.8,
                     add_colorbar = False)
ax[1,1].pcolormesh(ds[1].x_rho/1000,ds[1].y_rho/1000, ds[1].salt[t,0], 
                   vmin = 28, vmax = 35, cmap = cm)

ax[1,1].contour(ds[1].x_rho/1000,ds[1].y_rho/1000, ds[1].salt[t,0], 
                levels = np.arange(28,35.5,0.5), colors = 'k', linewidths = 0.8,
                add_colorbar = False)

ax[1,2].pcolormesh(ds[2].x_rho/1000,ds[2].y_rho/1000, ds[2].salt[t,0], 
                        vmin = 28, vmax = 35, cmap = cm)

ax[1,2].contour(ds[2].x_rho/1000,ds[2].y_rho/1000, ds[2].salt[t,0], 
                      levels = np.arange(28,36,0.5), colors = 'k', linewidths = 0.8,
                      add_colorbar = False)

fig.colorbar(cs, ax = ax[0,-1], label = r'Surface salinity [g kg$^{-1}$]', extend = 'both')
fig.colorbar(cs1, ax = ax[1,-1], label = r'Bottom salinity [g kg$^{-1}$]', extend = 'both')

for i in range(nrows):
    for j in range(ncols):
        ax[i,j].set_yticks(np.arange(0,100,20))
        ax[i,j].set_xticks(np.arange(0,100,20))
        ax[i,j].minorticks_on()
        ax[i,j].tick_params(which = 'both',bottom=True, top=True, left=True, right=True)
        ax[i,j].tick_params(which = 'both',labelbottom=True, labeltop=False, labelleft=True, labelright=False)
        ax[i,j].set_aspect('equal') # Set aspect to equal since the domain is square

for i in range(nrows):
    for j in range(ncols-1):
        ax[i,j+1].set_yticklabels('')

for i in range(nrows):
    ax[i,0].set_ylabel('Across-shore dist. [km]')
    
for j in range(ncols):
    ax[0,j].set_xticklabels('')
    ax[-1,j].set_xlabel('Along-shore dist. [km]')
    
ax[0,0].set_title('MPDATA')
ax[0,1].set_title('U3HC4')
ax[0,2].set_title('HSIMT')
#Add in subplot labels and specific contour lines
ax[0,0].text(3,91, '(a)', fontsize = 10, fontweight = 'heavy')
ax[0,1].text(3,91, '(b)', fontsize = 10, fontweight = 'heavy')
ax[0,2].text(3,91, '(c)', fontsize = 10, fontweight = 'heavy')
ax[0,0].text(40,87.5, '34.5', fontsize = 9)
ax[0,1].text(13,77, '34.5', fontsize = 9)
ax[0,1].text(50,90.25, '35', fontsize = 9)
ax[0,2].text(5,71, '34.5', fontsize = 9)

ax[1,0].text(3,91, '(d)', fontsize = 10, fontweight = 'heavy')
ax[1,1].text(3,91, '(e)', fontsize = 10, fontweight = 'heavy')
ax[1,2].text(3,91, '(f)', fontsize = 10, fontweight = 'heavy')
ax[1,0].text(11,40.5, '34.5', fontsize = 9)
ax[1,1].text(11,42, '34.5', fontsize = 9)
ax[1,1].text(40,60, '35', fontsize = 9)
ax[1,2].text(11,49, '34.5', fontsize = 9)

path = 'shelf_dx_500_tadv_dt_30_day_26.jpg'
# plt.savefig(path, dpi = 500, bbox_inches='tight')
# plt.clf()